In [3]:
from keras.models import load_model
model = load_model('S_model11.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              16000     
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                                 
 dense_2 (Dense)             (None, 300)               150300    
                                                                 
 dense_3 (Dense)             (None, 11)                3311      
                                                                 
Total params: 670,111
Trainable params: 670,111
Non-trainable params: 0
_________________________________________________________________


In [6]:
import cv2
import mediapipe as mp
import numpy as np
import math 
import time
import datetime



max_num_hands = 2
actions = {
    0: 'one',
    1: 'two',
    2: 'three',
    3: 'four', 
    4: 'five', 
    5: 'thumb',
    6: 'small', 
    7: 'Turn ON',
    8: 'Turn OFF',
    9: 'V',
    10: 'peace'
    
    
    
}

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=max_num_hands,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)




x_vector = np.array([-1,0,0])
x_vector = x_vector/np.linalg.norm(x_vector)

x_vector_2 = np.array([1,0,0])
x_vector_2 = x_vector_2/np.linalg.norm(x_vector_2)

y_vector = np.array([0,-1,0])
y_vector = y_vector/np.linalg.norm(y_vector)

x_mag = np.linalg.norm(x_vector)

cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture('gesture_test.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
record = False

while cap.isOpened():
    
    
    
    ret, img = cap.read()
    if not ret:
        continue
        
     
    
        
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    result = hands.process(img)

    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 3))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z]

            # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
            v = v2 - v1 # [20,3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # Convert radian to degree
            
            
            data = np.array([angle], dtype=np.float32)
            # Inference gesture
            y_pred = model.predict(data).squeeze() # 판단 logic

            idx = int(np.argmax(y_pred)) # argmax로 어떤 index인지 뽑아냄
            
            #
            #ret, results, neighbours, dist = knn.findNearest(data, 3)
            #idx = int(results[0][0])
            
            

            # Draw gesture result
            if idx in actions.keys():
                cv2.putText(img, text=actions[idx].upper(), org=(50,50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

            # Other gestures
            # cv2.putText(img, text=gesture[idx].upper(), org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
            
            
            if idx == 5:
                
                # tv= thumb_vector
                # ta= tubmb_angle
                # tc = thumb_cross
                
                tv = joint[1]-joint[0] 
                
                tv = tv / np.linalg.norm(tv) # 유닛벡터화
                
                
    
                ta = np.arccos(np.dot(tv,x_vector)) # 내적 후 arccos
                ta = np.degrees(ta)
           
                tc = np.cross(tv,x_vector) # 외적 
            
                # 각도 대비 값 조절 
                
                volume = (100/180) * ta  
                
                if tc[2]<= 0:
                    if idx in actions.keys():
                        cv2.putText(img, text="Volume :" + str(int(volume)), org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

            
            
            if idx == 6:
                
                
                             
                sv_2 = joint[18]-joint[17]
                
            
                
                sv_2 = sv_2 / np.linalg.norm(sv_2)
                
                
                
                # 좌
                b_1 = np.arccos(np.dot(sv_2,y_vector)) 
                b_1 = np.degrees(b_1)
                #print (a_1)
                #a_1 = int(a_1)
                
                b_2 = np.arccos(np.dot(sv_2,-(y_vector)))
                b_2 = np.degrees(b_2)
                
                min_angle_2 = b_1
                
                if min_angle_2 > b_2:
                    min_angle_2 = b_2
                    
                    
                
                if min_angle_2 == b_1: 
                    
                    if idx in actions.keys():
                        print("상")
                        cv2.putText(img, text="Channel up", org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

                elif min_angle_2 == b_2: 
                    
                    if idx in actions.keys():
                        print("하")
                        cv2.putText(img, text="Channel down", org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
 
                
                
            
            
            if idx == 9:
                
                # gv= 가위_vector
                # ga= 가위_angle
                # gc = 가위_cross
                
                gv = joint[6]-joint[5] 
                
                gv = gv / np.linalg.norm(gv)
                
                
                
                ga = np.arccos(np.dot(gv,y_vector))
                ga = np.degrees(ga)
                
                gc = np.cross(gv,y_vector)  
                
                gc2 = np.cross( -(x_vector),gv )
                # 각도 대비 값 조절 
                
                time = (12/90) * ga  
                
                
                if (gc[2] <= 0 and  gc2[2] <=  0 ):
                    if idx in actions.keys():
                        cv2.putText(img, text="Reservation Time :" + str(int(time)), org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
            
            if idx == 10:
                
                
                kv = joint[6]-joint[5] 
                
                
                
                sv = joint[18]-joint[17]
                
                
                
                pv = kv + sv 
                
                pv = pv / np.linalg.norm(pv)
                
                
                
                # 좌
                a_1 = np.arccos(np.dot(pv,x_vector)) 
                a_1 = np.degrees(a_1)
                #print (a_1)
                #a_1 = int(a_1)
                
                
                #상
                a_2 = np.arccos(np.dot(pv,y_vector))
                a_2 = np.degrees(a_2)
                #a_2 = int(a_2)
                
                
                #우
                a_3 = np.arccos(np.dot(pv,-(x_vector)))
                a_3 = np.degrees(a_3)
                #a_3 = int(a_3) 
                
                
                #하
                a_4 = np.arccos(np.dot(pv,-(y_vector)))
                a_4 = np.degrees(a_4)
                
                
                min_angle = a_1
                
                if min_angle > a_3:
                    min_angle = a_3
                    
                elif min_angle > a_2:
                    min_angle = a_2
                    
                elif min_angle > a_4:
                    min_angle = a_4    
             
            
                
                    
                    
                    
                
                if min_angle == a_1: 
                    
                    if idx in actions.keys():
                        print("좌")
                        cv2.putText(img, text="Left", org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

                elif min_angle == a_2: 
                    
                    if idx in actions.keys():
                        print("상")
                        cv2.putText(img, text="Top", org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
 
                    
                    
                elif min_angle == a_3: 
                    
                    if idx in actions.keys():
                        print("우")
                        cv2.putText(img, text="Right", org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
    
                
                elif min_angle == a_4: 
                    
                    if idx in actions.keys():
                        print("하")
                        cv2.putText(img, text="Down", org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

                
                
              
            
            
                
        
                           

    cv2.imshow('TEST', img)
    now = datetime.datetime.now().strftime("%d_%H-%M-%S")
    key = cv2.waitKey(33)

    if key == 27:
        break
    elif key == 26:
        print("캡쳐")
        cv2.imwrite(str(now) + ".png", img)
    elif key == 24:
        print("녹화 시작")
        record = True
        video = cv2.VideoWriter(str(now) + ".avi", fourcc, 20.0, (img.shape[1], img.shape[0]))
    elif key == 3:
        print("녹화 중지")
        record = False
        video.release()
        
    if record == True:
        print("녹화 중..")
        video.write(img)
capture.release()
cv2.destroyAllWindows()
    
  #  if cv2.waitKey(1) == ord('q'):
  #      break
  
  

1/1 [==============================] - 0s 26ms/step
상
1/1 [==============================] - 0s 29ms/step
상
1/1 [==============================] - 0s 28ms/step
상
1/1 [==============================] - 0s 27ms/step
상
1/1 [==============================] - 0s 34ms/step
상
1/1 [==============================] - 0s 32ms/step
상
1/1 [==============================] - 0s 31ms/step
상
1/1 [==============================] - 0s 29ms/step
상
1/1 [==============================] - 0s 33ms/step
상
1/1 [==============================] - 0s 27ms/step
상
1/1 [==============================] - 0s 28ms/step
우
1/1 [==============================] - 0s 27ms/step
우
1/1 [==============================] - 0s 28ms/step
우
1/1 [==============================] - 0s 26ms/step
우
1/1 [==============================] - 0s 24ms/step
우
1/1 [==============================] - 0s 25ms/step
우
1/1 [==============================] - 0s 25ms/step
우
1/1 [==============================] - 0s 24ms/step
우
1/1 [=======================

KeyboardInterrupt: 